In [4]:
import numpy as np
import pandas as pd




from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,AveragePooling2D, BatchNormalization,Dropout,Flatten,Dense
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
import tensorflow.keras
import pickle
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
with open('new_spectros.pkl', 'rb') as f:
     spectros= pickle.load(f)
    

In [ ]:
techno_arr = np.asarray(spectros[0])
pop_arr= np.asarray(spectros[1])
hip_arr = np.asarray(spectros[2])
garage_arr = np.asarray(spectros[3])
grime_arr=np.asarray(spectros[4])
rock_arr=np.asarray(spectros[5])
classical_arr =np.asarray(spectros[6])

In [ ]:
X=np.concatenate((techno_arr,pop_arr,hip_arr,garage_arr,grime_arr,rock_arr,classical_arr),axis=0)

In [ ]:
X_dim = X.reshape(X.shape[0],X.shape[1],X.shape[2],1)

In [ ]:
genre_dict = {
              'techno':0,
              'pop':1,
              'hip_hop':2,
              'garage':3,
              'grime':4,
              'rock':5,
              'classical':6
              }

In [ ]:
X_dim = X_dim / 255

In [ ]:
techno_y = np.repeat(0, techno_arr.shape[0]).reshape(-1,1)
pop_y = np.repeat(1, pop_arr.shape[0]).reshape(-1,1)
hip_hop_y =np.repeat(2,hip_arr.shape[0]).reshape(-1,1)
garage_y = np.repeat(3,garage_arr.shape[0]).reshape(-1,1)
grime_y = np.repeat(4,grime_arr.shape[0]).reshape(-1,1)
rock_y =np.repeat(5,rock_arr.shape[0]).reshape(-1,1)
classical_y =np.repeat(6,classical_arr.shape[0]).reshape(-1,1)

In [ ]:
ys = np.concatenate( (techno_y,pop_y,hip_hop_y,garage_y,grime_y,rock_y,classical_y),axis=0)

In [ ]:
y_try = ys.reshape(1,-1)

In [ ]:
#turn to one hot encoded
y_one_hot= to_categorical(y_try[0])
y_one_hot.shape

In [ ]:
X_train,X_test, y_train, y_test= train_test_split(X_dim,y_one_hot,test_size=0.1)

In [ ]:
X_train.resize(X_train.shape[0],X_train.shape[1],X_train.shape[2],3)
X_test_resized=np.resize(X_test,(X_test.shape[0],X_test.shape[1],X_test.shape[2],3))

In [ ]:
vgg16_conv = tf.keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False,
                                                   input_shape=(128, 644, 3))

avg = tf.keras.layers.GlobalAveragePooling2D()(vgg16_conv.output)
flat = tf.keras.layers.Flatten()(avg)
dense1 = tf.keras.layers.Dense(1024)(flat)
leak = tf.keras.layers.LeakyReLU(alpha=0.05)(dense1)
drop = tf.keras.layers.Dropout(0.5)(leak)
dense2 =tf.keras.layers.Dense(256)(drop)
leak2 = tf.keras.layers.LeakyReLU(alpha=0.05)(dense2)
drop2 = tf.keras.layers.Dropout(0.5)(leak2)
dense3 =tf.keras.layers.Dense(80)(drop2)
leak3 = tf.keras.layers.LeakyReLU(alpha=0.05)(dense3)
dense4= tf.keras.layers.Dense(40)(leak3)
leak4 = tf.keras.layers.LeakyReLU(alpha=0.05)(dense4)
output =  tf.keras.layers.Dense(7, activation='softmax', name='predictions')(leak4)
model = tf.keras.Model(inputs=vgg16_conv.input, outputs=output)

In [ ]:
# Setup a model checkpoint to save our best model


checkpoint = tf.keras.callbacks.ModelCheckpoint(
        '/content/drive/My Drive/Spotify_music_classification/models/vgg{epoch:02d}-{val_loss:.2f}.h5',
        
        verbose=1,
        save_best_only=True,
    )

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001)

In [ ]:
for layer in vgg16_conv.layers:
    layer.trainable=False

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    X_train,y_train,verbose=1,
    epochs=10,
    steps_per_epoch=3527,
    validation_data=(X_test_resized,y_test),shuffle=True,
    callbacks=[checkpoint])

In [ ]:
for layer in vgg16_conv.layers:
    layer.trainable=True

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
history = model.fit(
    X_train,y_train,verbose=1,
    epochs=10,
    steps_per_epoch=3527,
    validation_data=(X_test_resized,y_test),shuffle=True,
    callbacks=[checkpoint])